In [11]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pymongo
import sqlalchemy
from sqlalchemy import create_engine
import pymysql
import mysql.connector
from config import mongoConn, remote_db_endpoint, remote_db_port, remote_db_name, remote_db_user, remote_db_pwd

In [22]:
#connect to MySQLdb
pymysql.install_as_MySQLdb()
connstr = f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}"
engine = create_engine(connstr) 

In [26]:
#Test select statement
conn = engine.connect()
query = '''
    SELECT
        *
    FROM
        streamingservices

    '''
service_data = pd.read_sql(query, con=conn)
service_json = service_data.to_json(orient='records')

conn.close()
service_json

'[{"Service_ID":1,"Service_Name":"AcornTV","Service_Type":"SVOD"},{"Service_ID":2,"Service_Name":"HBO MAX","Service_Type":"SVOD"},{"Service_ID":3,"Service_Name":"HBO MAX","Service_Type":"SVOD"}]'

In [56]:
#MySQLdb insert connection
conn = pymysql.connect(host=f'{remote_db_endpoint}',
                             user=remote_db_user,
                             password=remote_db_pwd,
                             db=remote_db_name)
mycursor = conn.cursor()

In [25]:
#Insert records
sql = """
    INSERT INTO 
        streamingservices(Service_Name, Service_Type)
    VALUES 
        (%s, %s);
    """
val = ('HBO MAX', 'SVOD')
mycursor.execute(sql, val)

conn.commit()

In [88]:
#service scraper:
#connection
url = 'https://reelgood.com/services'
# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')
services = soup.select('ul.e2kiuu0 h3.eycwrsh3')

tVod = []
aVod = []
sVod = []
PopularVod = []
vodCount = 0

for service in services:
    if service.text == '':
        vodCount += 1
    if vodCount == 0:
        PopularVod.append(service.text)
    if vodCount == 4:
        aVod.append(service.text)
    if vodCount == 8:
        sVod.append(service.text)
    if vodCount == 12:
        tVod.append(service.text)
print(PopularVod)
        #print(vodCount)
    #print(service)

['Netflix', 'Prime Video', 'Hulu', 'Disney+', 'HBO MAX', 'Peacock', 'Apple TV+', 'Discovery+', 'Tubi', 'IMDb TV', 'Crackle', 'PlutoTV', 'Showtime', 'CBS All Access', 'Starz', 'Philo', 'fuboTV']


# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')
link = soup.select('.e1qyeclq5 a')[0]['href']
show_url = base_url + link
print(show_url)

In [101]:
#Delete records
sql = """
    DELETE FROM 
        streamingservices;
    """
mycursor.execute(sql) 
conn.commit()

In [102]:
#Insert tVod records
sql = """
    INSERT INTO 
        streamingservices(Service_Name, Service_Type, Popular)
    VALUES 
        (%s, %s, %s);
    """
for vod in tVod:
    POP = 0
    if vod in PopularVod:
        POP = 1
    if vod != '':
        val = (vod, 'TVOD', POP)
        mycursor.execute(sql, val) 
        
for vod in sVod:
    POP = 0
    if vod in PopularVod:
        POP = 1 
    if vod != '':
        val = (vod, 'SVOD', POP)
        mycursor.execute(sql, val)

for vod in aVod:
    POP = 0
    if vod in PopularVod:
        POP = 1 
    if vod != '':
        val = (vod, 'AVOD', POP)
        mycursor.execute(sql, val)
        
conn.commit()

In [104]:
def insert_service(vodList, VOD_TYPE):
    sql = """
        INSERT INTO 
            streamingservices(Service_Name, Service_Type, Popular)
        VALUES 
            (%s, %s, %s);
        """
    for vod in vodList:
        POP = 0
        if vod in PopularVod:
            POP = 1 
        if vod != '':
            val = (vod, VOD_TYPE, POP)
            mycursor.execute(sql, val)
    return True


# insert_service(aVod, 'AVOD')


True